In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# GLOBAL VARIABLES

dfModelSegSummaries = pd.DataFrame( [
    [0, 2019, 'data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/BY2019_Summary_SEGID.csv'      ],
    [0, 2023, 'data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/OY2023_Summary_SEGID.csv'      ],
    [0, 2028, 'data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/STIP2028_Summary_SEGID.csv'    ],
    [0, 2032, 'data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/FY2032_Summary_SEGID.csv'      ],
    [0, 2042, 'data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/FY2042_Summary_SEGID.csv'      ],
    [0, 2050, 'data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/FY2050_Summary_SEGID.csv'      ],
    [1, 2019, 'data/model-output/1_WF/v9_SE19_Net19_Summary_SEGID.csv'                         ],
    [1, 2023, 'data/model-output/1_WF/v9_SE23_Net23_Summary_SEGID.csv'                         ],
    [1, 2028, 'data/model-output/1_WF/v9_TIP_SE28_Net28_Summary_SEGID.csv'                     ],
    [1, 2032, 'data/model-output/1_WF/v9_RTP_SE32_Net32_Summary_SEGID.csv'                     ],
    [1, 2042, 'data/model-output/1_WF/v9_RTP_SE42_Net42_Summary_SEGID.csv'                     ],
    [1, 2050, 'data/model-output/1_WF/v9_RTP_SE50_Net50_Summary_SEGID.csv'                     ],
    [2, 2019, 'data/model-output/2_Cache/BY2019_Summary_SEGID.dbf'                             ],
    [2, 2023, 'data/model-output/2_Cache/BY2023_Summary_SEGID.dbf'                             ],
    [2, 2028, 'data/model-output/2_Cache/STIP_2028_Summary_SEGID.dbf'                          ],
    [2, 2032, 'data/model-output/2_Cache/FY2032_Summary_SEGID.dbf'                             ],
    [2, 2042, 'data/model-output/2_Cache/FY2042_Summary_SEGID.dbf'                             ],
    [2, 2050, 'data/model-output/2_Cache/FY2050_Summary_SEGID.dbf'                             ],
    [3, 2019, 'data/model-output/3_Dixie/Summary_SEGID_19.dbf'                                 ],
    [3, 2023, 'data/model-output/3_Dixie/Summary_SEGID_23.dbf'                                 ],
    [3, 2028, 'data/model-output/3_Dixie/Summary_SEGID_28.dbf'                                 ],
    [3, 2032, 'data/model-output/3_Dixie/Summary_SEGID_32.dbf'                                 ],
    [3, 2042, 'data/model-output/3_Dixie/Summary_SEGID_42.dbf'                                 ],
    [3, 2050, 'data/model-output/3_Dixie/Summary_SEGID_50.dbf'                                 ],
    [4, 2019, 'data/model-output/4_SuWsv2_2023-09-13_DRAFT/BY2019_Summary_SEGID.dbf'           ],
    [4, 2023, 'data/model-output/4_SuWsv2_2023-09-13_DRAFT/OY2023_Summary_SEGID.dbf'           ],
    [4, 2028, 'data/model-output/4_SuWsv2_2023-09-13_DRAFT/STIP2028_Summary_SEGID.dbf'         ],
    [4, 2032, 'data/model-output/4_SuWsv2_2023-09-13_DRAFT/FY2032_Summary_SEGID.dbf'           ],
    [4, 2042, 'data/model-output/4_SuWsv2_2023-09-13_DRAFT/FY2042_Summary_SEGID.dbf'           ],
    [4, 2050, 'data/model-output/4_SuWsv2_2023-09-13_DRAFT/FY2050_Summary_SEGID.dbf'           ],
    [5, 2019, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/BY2019_Summary_SEGID.dbf'  ],
    [5, 2023, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/OY2023_Summary_SEGID.dbf'  ],
    [5, 2028, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/STIP2028_Summary_SEGID.dbf'],
    [5, 2032, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/FY2032_Summary_SEGID.dbf'  ],
    [5, 2042, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/FY2042_Summary_SEGID.dbf'  ],
    [5, 2050, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/FY2050_Summary_SEGID.dbf'  ]
], columns=('modSubareaId','modYear','modSegSummaryFile'))

#display(dfModelSegSummaries)

# seg summary fields to be useds
use_fields = ['SEGID','DY_VOL','FT','ATYPENAME','LANES']

# segment shapefile with AADT and previous forecasts - copied from 'A:/1 - TDM/3 - Model Dev/1 - WF/1 - Official Release/v9x/v9.0/WF TDM v9.0 - official/1_Inputs/6_Segment/Segments_WF - 2023-08-01.shp'
fnSegmentsShapefile = 'data/segments/Final_Segments_BMP_EMP.shp'

# field for weekday factor
colFacWdAvg = 'FAC_WDAVG'
colFacWin = 'FAC_WIN'
colFacSpr = 'FAC_SPR'
colFacSum = 'FAC_SUM'
colFacFal = 'FAC_FAL'

# SHOULD WE ADJUST WEEKDAY FACTOR INTO FUTURE?? 

# base year adjustment
# segment shapefile with AADT and previous forecasts - copied from 'A:/1 - TDM/3 - Model Dev/1 - WF/1 - Official Release/v9x/v9.0/WF TDM v9.0 - official/1_Inputs/6_Segment/Segments_WF - 2023-08-01.shp'
sourceName = 'AADTHistory.xlsx'
baseYear = 2019

In [3]:
# Required libraries
from dbfread import DBF
import pandas as pd

# Create a list to store DataFrames read from each file
frames = []

# Iterate through the rows and read each file
for index, row in dfModelSegSummaries.iterrows():
    # Prepend path with 'dbfs:/'
    file_path = row['modSegSummaryFile']
    print(file_path)
    
    # Check the file extension
    if file_path.endswith('.dbf'):
        # Read the .dbf file with dbfread
        table = DBF(file_path)
        df = pd.DataFrame(iter(table))  # Convert the DBF table to a DataFrame
    elif file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    else:
        # If the file is neither .csv nor .dbf, skip this iteration
        print(f"Unsupported file format for file: {file_path}")
        continue

    df = df[use_fields]
    df['YEAR'     ] = row['modYear'     ] # Add modYear column
    df['SUBAREAID'] = row['modSubareaId'] # Add modSubareaId column
    frames.append(df)

# Concatenate all the frames into a single DataFrame
dfMdlVol = pd.concat(frames, ignore_index=True)

# If you're using Databricks, use 'display' to show the DataFrame
display(dfMdlVol)

data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/BY2019_Summary_SEGID.csv


C:\Users\bhereth\AppData\Local\Temp\ipykernel_3116\1412907378.py:20: DtypeWarning: Columns (153) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/OY2023_Summary_SEGID.csv


C:\Users\bhereth\AppData\Local\Temp\ipykernel_3116\1412907378.py:20: DtypeWarning: Columns (153) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/STIP2028_Summary_SEGID.csv


C:\Users\bhereth\AppData\Local\Temp\ipykernel_3116\1412907378.py:20: DtypeWarning: Columns (153) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/FY2032_Summary_SEGID.csv


C:\Users\bhereth\AppData\Local\Temp\ipykernel_3116\1412907378.py:20: DtypeWarning: Columns (153) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/FY2042_Summary_SEGID.csv
data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/FY2050_Summary_SEGID.csv
data/model-output/1_WF/v9_SE19_Net19_Summary_SEGID.csv
data/model-output/1_WF/v9_SE23_Net23_Summary_SEGID.csv
data/model-output/1_WF/v9_TIP_SE28_Net28_Summary_SEGID.csv
data/model-output/1_WF/v9_RTP_SE32_Net32_Summary_SEGID.csv
data/model-output/1_WF/v9_RTP_SE42_Net42_Summary_SEGID.csv
data/model-output/1_WF/v9_RTP_SE50_Net50_Summary_SEGID.csv
data/model-output/2_Cache/BY2019_Summary_SEGID.dbf
data/model-output/2_Cache/BY2023_Summary_SEGID.dbf
data/model-output/2_Cache/STIP_2028_Summary_SEGID.dbf
data/model-output/2_Cache/FY2032_Summary_SEGID.dbf
data/model-output/2_Cache/FY2042_Summary_SEGID.dbf
data/model-output/2_Cache/FY2050_Summary_SEGID.dbf
data/model-output/3_Dixie/Summary_SEGID_19.dbf
data/model-output/3_Dixie/Summary_SEGID_23.dbf
data/model-output/3_Dixie/Summary_SEGID_28.dbf
data/model-output/3_Dixie/Summary_SEGID_32.dbf
data/mode

,SEGID,DY_VOL,FT,ATYPENAME,LANES,YEAR,SUBAREAID
0,Cache_4013,659.7,6.0,Transition,1.0,2019,0
1,0006_000.0,126.7,2.0,Rural,1.0,2019,0
2,0006_000.7,213.3,2.0,Rural,1.0,2019,0
3,0006_016.0,76.6,2.0,Rural,1.0,2019,0
4,0006_046.0,76.6,2.0,Rural,1.0,2019,0
...,...,...,...,...,...,...,...
77542,3217_008.3,2708.0,3.0,Rural,2.0,2050,5
77543,3218_006.4,59.2,5.0,Rural,2.0,2050,5
77544,NO,461.9,6.0,Rural,2.0,2050,5
77545,UDOT_7008,462.4,4.0,Transition,2.0,2050,5


In [4]:
dfMdlVol[dfMdlVol['YEAR']==0]

,SEGID,DY_VOL,FT,ATYPENAME,LANES,YEAR,SUBAREAID


In [5]:
# GET WEEKDAY FACTORS

# read in segment shapefile
gdfSegments = gpd.read_file(fnSegmentsShapefile)
dfFac = gdfSegments[['SEGID', 'SUBAREAID', 'CO_FIPS', colFacWdAvg, colFacWin, colFacSpr, colFacSum, colFacFal]].copy()

display(dfFac)

,SEGID,SUBAREAID,CO_FIPS,FAC_WDAVG,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL
0,0006_000.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316
1,0006_000.7,0,27,0.9840,0.8495,1.0276,1.0914,1.0316
2,0006_016.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316
3,0006_046.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316
4,0006_060.2,0,27,0.9840,0.8495,1.0276,1.0914,1.0316
...,...,...,...,...,...,...,...,...
8717,0056_060.2,5,21,1.0924,0.9370,1.0104,1.0283,1.0243
8718,0056_060.6,5,21,1.0946,0.9653,1.0117,1.0135,1.0095
8719,0056_060.5,5,21,1.0946,0.9653,1.0117,1.0135,1.0095
8720,0056_060.4,5,21,1.0924,0.9370,1.0104,1.0283,1.0243


In [6]:
dfFac.loc[(dfFac['FAC_WDAVG']==0), 'checkZero'] = 1
dfFac.groupby(['SUBAREAID'],as_index = False).agg(numSegs=('SEGID','count'),numSegsWithFacZero=('checkZero','sum'))

,SUBAREAID,numSegs,numSegsWithFacZero
0,0,2577,0.0
1,1,4414,125.0
2,2,489,0.0
3,3,614,0.0
4,4,379,0.0
5,5,249,0.0


In [7]:
dfFac.replace('None', 0, inplace=True)
dfFac.fillna(0, inplace=True)

# Filter columns that start with "colFac"
fac_cols = [col for col in dfFac.columns if col.startswith('FAC_')]

dfFac[fac_cols] = dfFac[fac_cols].astype(float)

# Calculate the average of non-zero values for each "colFac" column by SUBAREAID
averages = {}
for col in fac_cols:
    averages[col] = dfFac[dfFac[col] != 0].groupby(['CO_FIPS'])[col].mean().to_dict()

print(averages)

def get_average_value(averages, column_name, co_fips):
    return averages.get(column_name, {}).get(co_fips, None)


{'FAC_WDAVG': {0: 1.0409166666666667, 1: 1.0060729166666667, 3: 1.0167221518987342, 5: 1.0633524489795918, 7: 1.0439963855421688, 9: 0.9685625, 11: 1.0930815352697096, 13: 0.9918183783783784, 15: 0.9869570247933884, 17: 0.9615215189873417, 19: 0.945358510638298, 21: 1.0189862903225808, 23: 1.0090029126213593, 25: 0.9667430769230769, 27: 0.9856744047619047, 29: 0.9840450000000001, 31: 0.9678333333333333, 33: 0.8772305555555555, 35: 1.0886187892632782, 37: 0.9676605263157896, 39: 0.9937018633540372, 41: 1.0115112195121951, 43: 1.0081473170731707, 45: 1.0446469325153376, 47: 1.0238054545454545, 49: 1.073645017921147, 51: 1.04265, 53: 1.0647285481239803, 55: 0.9635573770491803, 57: 1.0778987719298245}, 'FAC_WIN': {0: 0.8601333333333333, 1: 0.85385, 3: 0.8676534810126582, 5: 0.9028461224489797, 7: 0.8778006024096385, 9: 0.6991875, 11: 0.9377197095435685, 13: 0.834352972972973, 15: 0.7790024793388429, 17: 0.6245329113924051, 19: 0.572613829787234, 21: 0.8532302419354839, 23: 0.85542330097087

In [8]:
import numpy as np

# Your existing code for merging and filling NA
_df = pd.DataFrame.merge(dfFac, dfMdlVol, on=('SEGID','SUBAREAID'), how='inner')
_df.fillna(0, inplace=True)

# calculate AADT by subareaid
# 0-USTM: AADT already
# 2-5: divide by colWeekdayFactor

def custom_function(row):
    if row['SUBAREAID'] == 0:
        return row['DY_VOL']
    if row['SUBAREAID'] in [1,2,3,4,5]:
        if row[colFacWdAvg] > 0:
            return row['DY_VOL'] / row[colFacWdAvg]
        else:
            return row['DY_VOL'] / get_average_value(averages, colFacWdAvg, row['CO_FIPS']) # MAYBE REPLACE IN THE FUTURE WITH BETTER CODE
    else:
        return -1

_df['modAadt'] = _df.apply(custom_function, axis=1)


# Convert to integer as you had before
_df['modAadt'] = _df['modAadt'].astype(int)

dfModAadt = _df

# Display the DataFrame
display(dfModAadt)

,SEGID,SUBAREAID,CO_FIPS,FAC_WDAVG,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL,checkZero,DY_VOL,FT,ATYPENAME,LANES,YEAR,modAadt
0,0006_000.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,126.7,2.0,Rural,1.0,2019,126
1,0006_000.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,99.7,2.0,Rural,1.0,2023,99
2,0006_000.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,101.7,2.0,Rural,1.0,2028,101
3,0006_000.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,103.6,2.0,Rural,1.0,2032,103
4,0006_000.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,107.9,2.0,Rural,1.0,2042,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45252,1760_000.0,5,21,1.0924,0.9370,1.0104,1.0283,1.0243,0.0,4413.3,4.0,Suburban,2.0,2023,4040
45253,1760_000.0,5,21,1.0924,0.9370,1.0104,1.0283,1.0243,0.0,4850.8,4.0,Suburban,2.0,2028,4440
45254,1760_000.0,5,21,1.0924,0.9370,1.0104,1.0283,1.0243,0.0,5019.6,4.0,Suburban,2.0,2032,4595
45255,1760_000.0,5,21,1.0924,0.9370,1.0104,1.0283,1.0243,0.0,5603.1,4.0,Suburban,2.0,2042,5129


In [9]:
# SHOW WHERE FAC_WDAVG is 0... KEEP ZERO UNTIL FIGURE OUT WHAT TO DO!
dfModAadt[dfModAadt['YEAR']==0]


,SEGID,SUBAREAID,CO_FIPS,FAC_WDAVG,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL,checkZero,DY_VOL,FT,ATYPENAME,LANES,YEAR,modAadt


In [10]:
_df = pd.read_csv('intermediate/aadt.csv')
_df

,SEGID,YEAR,AADT,SOURCE
0,0006_000.0,1981,325,AADTHistory.xlsx
1,0006_000.0,1982,335,AADTHistory.xlsx
2,0006_000.0,1983,430,AADTHistory.xlsx
3,0006_000.0,1984,580,AADTHistory.xlsx
4,0006_000.0,1985,585,AADTHistory.xlsx
...,...,...,...,...
178372,3483_000.0,2017,990,AADTHistory.xlsx
178373,3483_000.0,2018,1030,AADTHistory.xlsx
178374,3483_000.0,2019,1042,AADTHistory.xlsx
178375,3483_000.0,2020,1041,AADTHistory.xlsx


In [11]:
# GET BASE YEAR OBSERVED

# read intermediate CSV with base year observed AADT
_df = pd.read_csv('intermediate/aadt.csv')

# filter by source and year
_df = _df[(_df['SOURCE']==sourceName) & (_df['YEAR']<=baseYear)]

# get max row of max year for each segid... incase there is no observed 2019 data, but maybe a earlier year
# Then, group by 'SEGID' and find the index of the maximum 'YEAR' for each group
idx = _df.groupby('SEGID')['YEAR'].idxmax()

# Use the index to filter the rows
_df = _df.loc[idx]

_df = _df[['SEGID','AADT']]
_df.rename(columns={'AADT':'obsAadt'}, inplace=True)

dfObsAadt_BaseYear = _df

display(dfObsAadt_BaseYear)

,SEGID,obsAadt
37,0006_000.0,415
77,0006_000.7,415
117,0006_016.0,415
157,0006_046.0,372
197,0006_060.2,372
...,...,...
178327,3468_000.0,2581
178339,3469_000.0,350
178344,3470_000.0,1724
178359,3478_000.0,911


In [12]:
# CALCULATE ADJUSTMENT FACTOR FROM BASE YEAR MODEL VS OBSERVED
dfModAadt_BaseYear = dfModAadt[dfModAadt['YEAR']==2019]

_df = pd.DataFrame.merge(dfModAadt_BaseYear[use_fields + ['modAadt']], dfObsAadt_BaseYear[['SEGID','obsAadt']], on='SEGID', how='left')
_df.fillna(0,inplace=True)

_df['aadtAdjFactor'] = _df['obsAadt'] - _df['modAadt']

_df.loc[(_df['obsAadt']==0), 'aadtAdjFactor'] = 0

display(_df[_df['modAadt']==0])

dfModAdjFactor = _df[['SEGID','aadtAdjFactor']]

display(dfModAdjFactor)


,SEGID,DY_VOL,FT,ATYPENAME,LANES,modAadt,obsAadt,aadtAdjFactor
189,0021_077.2,0.0,3.0,Transition,1.0,0,2605.0,2605.0
246,0025_000.0,0.0,4.0,Rural,1.0,0,179.0,179.0
247,0025_002.9,0.0,4.0,Rural,1.0,0,179.0,179.0
904,0122_000.0,0.0,4.0,Rural,1.0,0,77.0,77.0
918,0128_000.0,0.0,4.0,Rural,1.0,0,1055.0,1055.0
...,...,...,...,...,...,...,...,...
6789,Dixie_5091,0.0,3.0,Rural,2.0,0,0.0,0.0
6797,Dixie_5176,0.0,3.0,Rural,2.0,0,0.0,0.0
7023,2610_017.8,0.0,4.0,Rural,2.0,0,192.0,192.0
7258,1746_000.0,0.0,7.0,Rural,2.0,0,0.0,0.0


,SEGID,aadtAdjFactor
0,0006_000.0,289.0
1,0006_000.7,202.0
2,0006_016.0,339.0
3,0006_046.0,296.0
4,0006_060.2,292.0
...,...,...
7366,0015_056.8,10026.0
7367,0015_058.8,3062.0
7368,0056_060.2,-4564.0
7369,0056_060.5,-2000.0


In [13]:
# Function to round based on the given ranges
def custom_rounding(value):
    if 0 <= value < 100:
        return round(value / 10) * 10
    elif 100 <= value < 1000:
        return round(value / 50) * 50
    elif 1000 <= value < 10000:
        return round(value / 100) * 100
    elif 10000 <= value < 100000:
        return round(value / 500) * 500
    elif value >= 100000:
        return round(value / 1000) * 1000
    else:
        return value

In [14]:
# ADJUST ALL MODEL VOLUMES BASED ON BASE YEAR ADJ FACTOR

dfModWithAdj = pd.DataFrame.merge(dfModAadt, dfModAdjFactor, on='SEGID', how='left')
dfModWithAdj['aadtAdjFactor'].fillna(0,inplace=True)
dfModWithAdj['modForecast'] = (dfModWithAdj['modAadt'] + dfModWithAdj['aadtAdjFactor']).apply(custom_rounding)
dfModWithAdj

# remove 2019 since it is base
#dfModWithAdjNo2019 = dfModWithAdj[dfModWithAdj['YEAR']>2019]

# sort
#dfModWithAdjNo2019 = dfModWithAdjNo2019.sort_values(by=['SEGID', 'YEAR'], ascending=[True, True])

#dfModWithAdjNo2019

,SEGID,SUBAREAID,CO_FIPS,FAC_WDAVG,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL,checkZero,DY_VOL,FT,ATYPENAME,LANES,YEAR,modAadt,aadtAdjFactor,modForecast
0,0006_000.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,126.7,2.0,Rural,1.0,2019,126,289.0,400.0
1,0006_000.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,99.7,2.0,Rural,1.0,2023,99,289.0,400.0
2,0006_000.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,101.7,2.0,Rural,1.0,2028,101,289.0,400.0
3,0006_000.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,103.6,2.0,Rural,1.0,2032,103,289.0,400.0
4,0006_000.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,107.9,2.0,Rural,1.0,2042,107,289.0,400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45252,1760_000.0,5,21,1.0924,0.9370,1.0104,1.0283,1.0243,0.0,4413.3,4.0,Suburban,2.0,2023,4040,-3042.0,1000.0
45253,1760_000.0,5,21,1.0924,0.9370,1.0104,1.0283,1.0243,0.0,4850.8,4.0,Suburban,2.0,2028,4440,-3042.0,1400.0
45254,1760_000.0,5,21,1.0924,0.9370,1.0104,1.0283,1.0243,0.0,5019.6,4.0,Suburban,2.0,2032,4595,-3042.0,1600.0
45255,1760_000.0,5,21,1.0924,0.9370,1.0104,1.0283,1.0243,0.0,5603.1,4.0,Suburban,2.0,2042,5129,-3042.0,2100.0


In [15]:
# check if base year adjusted equals base year observed

_df1 = dfModWithAdj[dfModWithAdj['YEAR']==baseYear]
_df2 = dfObsAadt_BaseYear

_df3 = pd.DataFrame.merge(_df1, _df2, on='SEGID')

_df3['ModVsObs'] = _df3['modForecast'] - _df3['obsAadt']

display(_df3[_df3['ModVsObs']!=0])


,SEGID,SUBAREAID,CO_FIPS,FAC_WDAVG,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL,checkZero,DY_VOL,FT,ATYPENAME,LANES,YEAR,modAadt,aadtAdjFactor,modForecast,obsAadt,ModVsObs
0,0006_000.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,126.7,2.0,Rural,1.0,2019,126,289.0,400.0,415,-15.0
1,0006_000.7,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,213.3,2.0,Rural,1.0,2019,213,202.0,400.0,415,-15.0
2,0006_016.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,76.6,2.0,Rural,1.0,2019,76,339.0,400.0,415,-15.0
3,0006_046.0,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,76.6,2.0,Rural,1.0,2019,76,296.0,350.0,372,-22.0
4,0006_060.2,0,27,0.9840,0.8495,1.0276,1.0914,1.0316,0.0,80.4,2.0,Rural,1.0,2019,80,292.0,350.0,372,-22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6619,0015_056.8,5,21,1.0654,0.9401,1.0094,1.0374,1.0131,0.0,23216.0,35.0,Suburban,4.0,2019,21790,10026.0,32000.0,31816,184.0
6620,0015_058.8,5,21,0.9701,0.8713,1.0026,1.1123,1.0138,0.0,27277.1,35.0,Suburban,4.0,2019,28117,3062.0,31000.0,31179,-179.0
6621,0056_060.2,5,21,1.0924,0.9370,1.0104,1.0283,1.0243,0.0,19451.9,2.0,Suburban,4.0,2019,17806,-4564.0,13000.0,13242,-242.0
6622,0056_060.5,5,21,1.0946,0.9653,1.0117,1.0135,1.0095,0.0,20705.4,2.0,Suburban,4.0,2019,18915,-2000.0,17000.0,16915,85.0


In [16]:
# export final file
dfModWithAdj.to_csv('intermediate/model-forecasts.csv', index=False)

In [17]:
dfModWithAdj[dfModWithAdj['SEGID']=='WFRC_8019']

,SEGID,SUBAREAID,CO_FIPS,FAC_WDAVG,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL,checkZero,DY_VOL,FT,ATYPENAME,LANES,YEAR,modAadt,aadtAdjFactor,modForecast
35154,WFRC_8019,1,11,1.1568,0.9529,1.0112,1.0254,1.0105,0.0,24063.8,33.0,Suburban,4.0,2028,20802,0.0,21000.0
35155,WFRC_8019,1,11,1.1568,0.9529,1.0112,1.0254,1.0105,0.0,28462.7,33.0,Suburban,4.0,2032,24604,0.0,24500.0
35156,WFRC_8019,1,11,1.1568,0.9529,1.0112,1.0254,1.0105,0.0,31901.0,33.0,Suburban,4.0,2042,27576,0.0,27500.0
35157,WFRC_8019,1,11,1.1568,0.9529,1.0112,1.0254,1.0105,0.0,36437.5,33.0,Suburban,4.0,2050,31498,0.0,31500.0
